### DEEP LEARNING MODEL FOR GEAR CLASSIFCATION

In [3]:
import os
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
import pickle
import json
from IPython.display import Image, display

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 

In [4]:
with open(r"output.pickle", "rb") as input_file:
    data = pickle.load(input_file)
    
labels = []
features = []

for k, v in data.items():
    for value in v:
        features.append(value)
        labels.append(k)
    
labels = np.asarray(labels)
features = np.asarray(features)
print (features.shape)
print (labels.shape)


(2122, 128, 128, 3)
(2122,)


In [37]:
from sklearn.model_selection import train_test_split
from keras.utils import np_utils 
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.15)


# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)

encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(encoded_Y)

print (y_train.shape)

# y_test = to_categorical(y_test)


(1803, 12)


In [38]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Lambda, Flatten, Dropout

filters = 64
kernel_size = (5,5)

model = Sequential()
model.add(Lambda(lambda x: x/255 - 1., input_shape=(128, 128, 3)))

model.add(Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', activation = 'relu' ))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'))
model.add(Dropout(0.3))
model.add(Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', activation = 'relu' ))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'))

model.add(Flatten())
model.add(Dense(12, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3, validation_split = 0.1)
score = model.evaluate(X_test, y_test)

print (score)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 124, 124, 64)      4864      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 58, 58, 64)        102464    
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 29, 29, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 53824)             0         
__________

In [41]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(X_test)

y_test = np.argmax(y_test, axis= 1)
predictions = np.argmax(predictions, axis = 1)

print (y_test.shape)

confusion_matrix(y_test, predictions)

(319,)


array([[11,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 18,  0,  0,  2,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0, 44,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 12,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 35,  0,  0,  1,  1,  0,  1,  0],
       [ 1,  0,  0,  0,  1, 54,  0,  0, 11,  1,  0,  0],
       [ 1,  0,  0,  0,  1,  0, 29,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  5,  0,  0, 28,  0,  0,  0],
       [ 0,  0,  2,  0,  1,  2,  0,  0,  0,  1,  1,  1],
       [ 2,  0,  0,  0,  1,  0,  0,  0,  0,  0, 20,  2],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0, 15]])

In [42]:
print (np.unique(labels))

['axes' 'boots' 'carabiners' 'crampons' 'gloves' 'hardshell_jackets'
 'harnesses' 'helmets' 'insulated_jackets' 'pulleys' 'rope' 'tents']


In [45]:
from keras.models import load_model

model.save('my_model.h5')